In [ ]:
from numpy import load
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = load('/content/drive/MyDrive/PathMNIST/pathmnist.npz')
lst = data.files
print(len(lst))
print("Adatok:")
for i in range(6):
    print(lst[i],':',len(data[lst[i]]), "db")

6
Adatok:
train_images : 89996 db
val_images : 10004 db
test_images : 7180 db
train_labels : 89996 db
val_labels : 10004 db
test_labels : 7180 db


In [ ]:
target = [] #---> 0,1,2,3
image_data = [] #---->array

target = np.concatenate((data[lst[3]].flatten(), data[lst[4]].flatten(),data[lst[5]].flatten()))
image_data = np.concatenate((data[lst[0]], data[lst[1]],data[lst[2]]))

print(len(target))
print(len(image_data))

data_size = len(target)

107180
107180


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_data,target, test_size=0.2)

train_size = len(X_train)
test_size = len(X_test)

In [ ]:
#print(X_train[23])
X_train = X_train.reshape(train_size,3,28,28)
#print(X_train[23])
X_test = X_test.reshape(test_size,3,28,28)
print(len(X_train), len(X_test))


85744 21436


In [ ]:

from torchvision import transforms


X_train = torch.Tensor(X_train).reshape(-1,3,28,28)
#X_train = transforms.ToTensor()

#X_train = X_train/255.0
#X_train = torch.from_numpy(np.array(X_train, dtype=np.int))
#y_train = torch.Tensor(y_train)
#X_test = torch.from_numpy(np.array(X_test, dtype=np.int))
X_test = torch.Tensor(X_test).reshape(-1,3,28,28)
#X_test = transforms.ToTensor()
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

#X_test = X_test/255.0
#y_test = torch.Tensor(y_test)



#print(y_train[0])
#print(y_test.size())


print(y_train[10])
#y_train.type(torch.LongTensor)

tensor(0)


In [ ]:
train_set = []
test_set = []
bb = []

for i in range(85744) :
  bb = []
  bb.append(X_train[i])
  bb.append(y_train[i])
  train_set.append(bb)

print(len(train_set))
print(len(train_set[10]))
#print(train_set[10])

for i in range(21436) :
  bb = []
  bb.append(X_test[i])
  bb.append(y_test[i])
  test_set.append(bb)



print(len(test_set))
print(type((test_set[10][1])))

85744
2
21436
<class 'torch.Tensor'>


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_set,
    batch_size=32,
    num_workers=4,
    shuffle=True,
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size = 32,
    num_workers = 4,
    shuffle = True,
)

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv = nn.Conv2d(3, 18, kernel_size = 3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(18 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, 9) #itt az osztályok száma
        

    
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.pool(x)
        x = x.view(-1, 18 * 16 * 16)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    
net = Net()
print(net)

Net(
  (conv): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=9, bias=True)
)


In [ ]:
#importáljuk a már kész NN-t

from torchvision.models import squeezenet1_0

model = squeezenet1_0(pretrained=True)
print(model)

Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-a815701f.pth



SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
  

In [ ]:
#módosítjuk az utolsó rétegét a saját layeremre

n_classes = 9

model.num_classes = n_classes
model.classifier[1] = nn.Conv2d(512, n_classes, kernel_size=(1,1), stride=(1,1))

In [ ]:
def acc_count():
  correct, total = 0, 0
  predictions = []
  model.eval()
  for i, data in enumerate(test_loader, 0):
      inputs, labels = data
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      predictions.append(outputs)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  print('Háló pontossága a test képeken %d %%' % (100 * correct / total))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

for epoch in range(5):
    for i, data in enumerate(train_loader, 0):
        # Get the inputs
        inputs, labels = data
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(epoch,'. epoch')
    acc_count()
print('Finished Training')

0 . epoch
Háló pontossága a test képeken 47 %
1 . epoch
Háló pontossága a test képeken 61 %
2 . epoch
Háló pontossága a test képeken 61 %
3 . epoch
Háló pontossága a test képeken 65 %
4 . epoch
Háló pontossága a test képeken 64 %
Finished Training


In [16]:
acc_count()

for epoch in range(20):
    for i, data in enumerate(train_loader, 0):
        # Get the inputs
        inputs, labels = data
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(epoch,'. epoch')
    acc_count()
print('Finished Training')

Háló pontossága a test képeken 64 %
0 . epoch
Háló pontossága a test képeken 67 %
1 . epoch
Háló pontossága a test képeken 68 %
2 . epoch
Háló pontossága a test képeken 69 %
3 . epoch
Háló pontossága a test képeken 71 %
4 . epoch
Háló pontossága a test képeken 71 %
5 . epoch
Háló pontossága a test képeken 70 %
6 . epoch
Háló pontossága a test képeken 72 %
7 . epoch
Háló pontossága a test képeken 73 %
8 . epoch
Háló pontossága a test képeken 73 %
9 . epoch
Háló pontossága a test képeken 63 %
10 . epoch
Háló pontossága a test képeken 73 %
11 . epoch
Háló pontossága a test képeken 73 %
12 . epoch
Háló pontossága a test képeken 70 %
13 . epoch
Háló pontossága a test képeken 74 %
14 . epoch
Háló pontossága a test képeken 73 %
15 . epoch
Háló pontossága a test képeken 73 %
16 . epoch
Háló pontossága a test képeken 72 %
17 . epoch
Háló pontossága a test képeken 74 %
18 . epoch
Háló pontossága a test képeken 74 %
19 . epoch
Háló pontossága a test képeken 74 %
Finished Training
